In [1]:
import random
import math # cos() for Rastrigin
import copy # array-copying convenience
import sys	 # max float
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt



In [2]:
# read the csv file
df = pd.read_csv('dataIntel.csv')


In [3]:
df.sample(10)

,Unnamed: 0,DATE,TIME,EPOCH,NODEID,TEMPERATURE,HUMIDITY,LIGHT,VOLTAGE,x_coord,y_coord
44,44,2004-02-28,01:08:45.992568,55070,45,18.9398,45.4732,0.92,2.42416,16,19
91,91,2004-02-28,01:08:45.992615,2907,92,18.1460,40.9730,0.92,2.65143,46,24
98,98,2004-02-28,01:08:45.992622,24449,99,31.5132,26.6080,1847.36,2.69964,31,64
87,87,2004-02-28,01:08:45.992611,41118,88,23.9672,34.3621,1494.08,2.51661,19,82
17,17,2004-02-28,01:08:45.992541,26200,18,21.6936,40.5004,114.08,2.57108,50,12
50,50,2004-02-28,01:08:45.992574,25675,51,21.0370,46.0655,1.84,2.59354,69,68
0,0,2004-02-28,01:08:45.992524,22,1,19.1456,38.9401,45.08,2.68742,10,7
79,79,2004-02-28,01:08:45.992603,12924,80,25.0354,28.2948,713.92,2.71196,15,3
73,73,2004-02-28,01:08:45.992597,21248,74,24.1534,35.6802,441.60,2.63964,100,71
66,66,2004-02-28,01:08:45.992590,14310,67,20.0080,37.1277,0.46,2.53812,28,14


In [4]:
x = []
y = []
for i in range(len(df)):
    x.append(df['x_coord'][i])
    y.append(df['y_coord'][i])

In [5]:
# global variables
global xc,yc,A,xbs,ybs,NCH,L,Reg,efs,emp,dth,N,Eda,Eelc # global variables
xc = 50 # centroid x coordinate
yc = 50 # centroid y coordinate
A = 0.5
xbs = 0 # base station x coordinate
ybs = 0 # base station y coordinate
NCH = 10 # number of cluster heads
L = 4000 # packet size (4000 bit)
Reg = 100 # region size (100m*100m)
efs = 10 * pow(10,-12) # energy factor for free space enviornment (10pj/bit/m^2)
emp = 0.0013 * pow(10,-12) # energy factor for multi space environment
dth = 88 # threshold distance
N = 100 #number of nodes
Eda = 5 * pow(10,-9) #energy consumption per bit during the data aggregation operation (5nj/bit/signal)
Eelc = 50 * pow(10,-9) #  energy consumption during 1-bit transmission or reception (50nj/bit/signal)





In [6]:
# Node Class
class Node:

    # constructor
    def __init__(self,x,y,id):
        self.x = x
        self.y = y
        self.id = id
        self.ein = 0.5
        self.On = self.findOn()
        self.isDirect = True if self.On == 1 else False
    
    def position(self):
        print(self.x,self.y)
    
    def dn2bs(self):   # node to base station distance
        return math.sqrt((self.x - xbs)**2 + (self.y - ybs)**2)
    
    def dn2n(self,xi,yi):  #node to particular node distance
        return math.sqrt((self.x - xi)**2 + (self.y - yi)**2)

    def dn2c(self):   # node to centroid distance
        return math.sqrt((self.x - xc)**2 + (self.y - yc)**2)

    # calculate On
    def findOn(self):
        Emean = 0.1
        Eres = 0
        if(self.dn2bs() < dth):
            Eres = self.ein - (L*Eelc + L*efs*self.dn2bs()**2)
        else:
            Eres = self.ein - (L*Eelc + L*emp*self.dn2bs()**4)
        
        if(Eres >= Emean and self.dn2bs() < dth):
            return 1
        else:
            return 0

    



In [7]:
Nodes = []
for i in range(0,100):
    obj = Node(x[i],y[i],i)
    Nodes.append(obj)

In [8]:
# Calculate Energies
def findResidualTotal(Nodes):  # Residual Energy of all nodes
    Eres = []
    print(type(Nodes))
    for i in range(0,len(Nodes)):
        Eres.append(Nodes[i].ein - (L*Eelc + L*efs*Nodes[i].dn2bs()**2))
    return Eres

def findResidual(Node):  # Residual Energy of a particular node
    Eres = Node.ein - (L*Eelc + L*efs*Node.dn2bs()**2)
    return Eres


def findETx(d):  # energy consumption for transmission
    if(d < dth):
        return L*Eelc + L*efs*d**2
    else:
        return L*Eelc + L*emp*d**4

def findERx(): # energy consumption for reception
    return L*Eelc








In [9]:
Eres = findResidual(Nodes[3])
Eres

0.4994288

In [10]:
# Number of Alive Nodes
def findAliveNodes(Nodes):
    count = 0
    for i in range(0,len(Nodes)):
        if(Nodes[i].On == 0 and Nodes[i].ein > 0):
            count += 1
    return count

In [11]:
Na = findAliveNodes(Nodes)
print(Na)

33


In [12]:
#   Count of Direct Nodes
def findDirectNodes(Nodes):
    count = 0
    for i in range(0,len(Nodes)):
        if(Nodes[i].On == 1):
            count += 1
    return count

In [13]:
ds = findDirectNodes(Nodes)
print(ds)

67


In [14]:
# Optimum Number of K
dbs = 50
Kopt = (math.sqrt(Na/(2*math.pi)))*(math.sqrt(efs/emp))*(Reg/dbs**2) # optimum number of Cluster Heads
Kopt  

8.039987081530032

In [15]:
Nc = int(Na/ Kopt) # optimum number of nodes in each cluster
Nc


4

In [16]:
# find x1
def findx1(Node):
    return findResidual(Node)/Node.ein

# dmax
def findDmax(Nodes):
    d =[]
    for i in range(0,Nc):
        d.append(Nodes[i].dn2c())
    return max(d)

dmax = findDmax(Nodes)

# find x2
def findX2(Node):
    return (dmax - Node.dn2c())/dmax

# find x2dash
def findX2dash(Node):
    return (Node.dn2c()-dmax)/Node.dn2c()





In [17]:
# FCH Function
dmax = findDmax(Nodes)
def findFCH(Nodes,dmax):
    FCH = []
    alpha = 0.8
    ans = 0
    for i in range(0,100):
        if(Nodes[i].isDirect == False):
            if(Nodes[i].dn2c() < dmax):
                ans = alpha*findx1(Nodes[i]) + (1-alpha)*findX2(Nodes[i])
                FCH.append(ans)
            elif(Nodes[i].dn2c() == dmax):
                ans = findx1(Nodes[i])
                FCH.append(ans)
            else:
                ans = alpha*findx1(Nodes[i]) + (1-alpha)*findX2dash(Nodes[i])
                FCH.append(ans)
        else:
            FCH.append(-1)
        
    return FCH


    

In [18]:
FCH = findFCH(Nodes,dmax)
print(FCH)



[-1, -1, -1, 0.9988576, -1, -1, 0.8383011063782272, -1, -1, -1, 0.8020709379216264, 0.8801986983161102, -1, 0.8775540633087218, -1, -1, 0.9150445120492653, -1, -1, -1, 0.8463577617290194, 0.9374088239144324, 0.8028442144494727, -1, -1, -1, -1, 0.8146684388541422, 0.8407766555511752, -1, 0.8769650330349845, 0.89901632, -1, -1, -1, -1, 0.8556716937178307, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0.905798559379694, -1, 0.9150445120492653, -1, 0.9064132277340667, -1, 0.8291725202575378, -1, -1, -1, -1, -1, -1, 0.937486005323951, 0.8679705842071402, -1, -1, -1, -1, 0.8993648621009209, -1, -1, 0.8590363979430183, -1, -1, 0.8245921692409958, -1, 0.8481613010679742, 0.8035340904117767, -1, 0.875902883123951, -1, -1, 0.8062452633832865, -1, 0.8304570428128965, -1, -1, 0.8246619930842531, -1, -1, -1, 0.8256016174715288, -1, -1, 0.8512330772143748, 0.8594467014432547, -1, -1, -1, -1, 0.849063215269156]


In [19]:
C = copy.deepcopy(FCH)
FCH.sort(reverse=True)
FCH[0:8]
CH = [FCH[0:8]]
print(CH)
print(C[0:8])


[[0.9988576, 0.937486005323951, 0.9374088239144324, 0.9150445120492653, 0.9150445120492653, 0.9064132277340667, 0.905798559379694, 0.8993648621009209]]
[-1, -1, -1, 0.9988576, -1, -1, 0.8383011063782272, -1]


In [20]:
count = 0
for i in range(0,len(FCH)):
    if(FCH[i] == -1):
        count += 1
print(count)

67


In [21]:
a = Nodes[6].y
print(a)

51


In [22]:
df2 = df['NODEID']
df2 = pd.DataFrame(df2)

In [23]:
df2['x_coord'] = x
df2['y_coord'] = y
df2['FCH'] = C
df2.tail(50)

,NODEID,x_coord,y_coord,FCH
50,51,69,68,0.915045
51,52,28,41,-1.000000
52,53,78,57,0.906413
53,54,5,5,-1.000000
54,55,92,82,0.829173
55,56,72,34,-1.000000
56,57,75,33,-1.000000
57,58,49,5,-1.000000
58,59,33,68,-1.000000
59,60,35,37,-1.000000
